In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
import psycopg2
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [2]:
guns_2016 = pd.read_csv('2016_gun_violence.csv')
guns_2017 = pd.read_csv('2017_gun_violence.csv')
population = pd.read_csv('population_clean.csv')
poverty_2016 = pd.read_csv('poverty_2016_clean.csv')
poverty_2017 = pd.read_csv('poverty_2017_clean.csv')

In [32]:
population = pd.read_csv('population_clean.csv')


In [4]:
poverty_2016 = poverty_2016.rename(columns={'Unnamed: 0': 'id'})
poverty_2017 = poverty_2017.rename(columns={'Unnamed: 0': 'id'})

In [33]:
population = population.rename(columns={'Unnamed: 0': 'id'})

In [6]:
guns_2017 = guns_2017.reset_index(inplace=False)
guns_2017 = guns_2017.rename(columns={'index': 'id'})
guns_2016 = guns_2016.reset_index(inplace=False)
guns_2016 = guns_2016.rename(columns={'index': 'id'})


In [12]:
poverty_2016 = poverty_2016.rename(columns={'Location': 'state'})
poverty_2017 = poverty_2017.rename(columns={'Location': 'state'})

In [14]:
poverty_2016['year'] = 2016
poverty_2017['year'] = 2017

In [15]:
guns_2016['year'] = 2016
guns_2017['year'] = 2017

In [19]:
poverty_2016 = poverty_2016.iloc[1:]

In [22]:
poverty_2017 = poverty_2017.iloc[1:]

In [23]:
print(guns_2016.head())
print(guns_2017.head())
print(poverty_2016.head())
print(poverty_2017.head())
print(population.head())

   id          state  n_killed  n_injured  year
0   0        Alabama       488        761  2016
1   1          Maine        18         37  2016
2   2       Maryland       429        748  2016
3   3  Massachusetts       112        468  2016
4   4       Michigan       385        700  2016
   id         state  n_killed  n_injured  year
0   0       Vermont         9         11  2017
1   1   Connecticut        89        293  2017
2   2        Oregon        99        170  2017
3   3  Pennsylvania       635       1188  2017
4   4      Colorado       199        300  2017
   id       state  White  Black  Hispanic  \
1   1     Alabama  0.121  0.279     0.319   
2   2      Alaska  0.068  0.000     0.178   
3   3     Arizona  0.110  0.211     0.238   
4   4    Arkansas  0.138  0.288     0.255   
5   5  California  0.096  0.223     0.190   

   Asian/Native Hawaiian and Pacific Islander  American Indian/Alaska Native  \
1                                       0.118                          0.126   

In [38]:
population = population.drop([30,49])

In [40]:
merged_df = pd.merge(pd.merge(pd.merge(pd.merge(poverty_2016,poverty_2017,on='state'),population,on='state'), guns_2016, on='state'), guns_2017, on='state')

In [42]:
merged_2016 = pd.merge(pd.merge(poverty_2016,guns_2016,on='state'),population ,on='state')

In [44]:
merged_2016 = merged_2016.drop(['id_y', 'year_y', 'id'], axis=1)
merged_2016 = merged_2016.rename(columns={'id_x': 'id', 'rank': 'population_rank', 'Total': 'total_poverty', 'year_x': 'year'})

In [45]:
merged_2016.head()

,id,state,White,Black,Hispanic,Asian/Native Hawaiian and Pacific Islander,American Indian/Alaska Native,Multiple Races,total_poverty,year,n_killed,n_injured,population_rank,population,density_per_sq_mi
0,1,Alabama,0.121,0.279,0.319,0.118,0.126,0.251,0.173,2016,488,761,24,4887681,97.4271
1,2,Alaska,0.068,0.000,0.178,0.000,0.274,0.104,0.106,2016,88,103,49,735139,1.2694
2,3,Arizona,0.110,0.211,0.238,0.136,0.360,0.155,0.165,2016,277,294,14,7158024,66.2016
3,4,Arkansas,0.138,0.288,0.255,0.129,0.263,0.254,0.172,2016,190,351,34,3009733,58.3059
4,5,California,0.096,0.223,0.190,0.113,0.226,0.132,0.144,2016,1265,1896,1,39461588,254.2929


In [46]:
merged_2017 = pd.merge(pd.merge(poverty_2017,guns_2017,on='state'),population ,on='state')

In [47]:
merged_2017 = merged_2017.drop(['id_y', 'year_y', 'id'], axis=1)
merged_2017 = merged_2017.rename(columns={'id_x': 'id', 'rank': 'population_rank', 'Total': 'total_poverty', 'year_x': 'year'})

In [48]:
merged_2017.head()

,id,state,White,Black,Hispanic,Asian/Native Hawaiian and Pacific Islander,American Indian/Alaska Native,Multiple Races,total_poverty,year,n_killed,n_injured,population_rank,population,density_per_sq_mi
0,1,Alabama,0.117,0.269,0.306,0.193,0.000,0.239,0.168,2017,544,856,24,4887681,97.4271
1,2,Alaska,0.083,0.000,0.000,0.000,0.246,0.112,0.117,2017,69,70,49,735139,1.2694
2,3,Arizona,0.101,0.230,0.197,0.109,0.360,0.143,0.147,2017,289,298,14,7158024,66.2016
3,4,Arkansas,0.130,0.278,0.237,0.150,0.284,0.163,0.162,2017,216,437,34,3009733,58.3059
4,5,California,0.089,0.204,0.174,0.103,0.177,0.121,0.132,2017,1423,1953,1,39461588,254.2929


In [49]:
merged_df = pd.concat([merged_2016, merged_2017])

In [53]:
merged_df = merged_df.reset_index()

In [55]:
merged_df = merged_df.drop(['index', 'id'], axis=1)

In [56]:
import numpy as np

In [57]:
index = np.arange(1,101,1)

In [60]:
index.nunique()

AttributeError: 'numpy.ndarray' object has no attribute 'nunique'

In [61]:
merged_df['id'] = index

In [63]:
merged_df.count()

state                                         100
White                                         100
Black                                         100
Hispanic                                      100
Asian/Native Hawaiian and Pacific Islander    100
American Indian/Alaska Native                 100
Multiple Races                                100
total_poverty                                 100
year                                          100
n_killed                                      100
n_injured                                     100
population_rank                               100
population                                    100
density_per_sq_mi                             100
id                                            100
dtype: int64

In [64]:
first_column = merged_df.pop('id')
merged_df.insert(0, 'id', first_column)

In [67]:
second_column = merged_df.pop('year')
  
merged_df.insert(1, 'year', second_column)

In [68]:
merged_df.head()

,id,year,state,White,Black,Hispanic,Asian/Native Hawaiian and Pacific Islander,American Indian/Alaska Native,Multiple Races,total_poverty,n_killed,n_injured,population_rank,population,density_per_sq_mi
0,1,2016,Alabama,0.121,0.279,0.319,0.118,0.126,0.251,0.173,488,761,24,4887681,97.4271
1,2,2016,Alaska,0.068,0.000,0.178,0.000,0.274,0.104,0.106,88,103,49,735139,1.2694
2,3,2016,Arizona,0.110,0.211,0.238,0.136,0.360,0.155,0.165,277,294,14,7158024,66.2016
3,4,2016,Arkansas,0.138,0.288,0.255,0.129,0.263,0.254,0.172,190,351,34,3009733,58.3059
4,5,2016,California,0.096,0.223,0.190,0.113,0.226,0.132,0.144,1265,1896,1,39461588,254.2929


In [69]:
connection_string = "postgres:postgres@localhost:5432/guns_db"
engine = create_engine(f'postgresql://{connection_string}')

In [71]:
engine.table_names()

<ipython-input-71-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['2016poverty',
 '2016_gun_violence',
 '2017_gun_violence',
 'totalpopulation',
 '2017poverty',
 'merged_data']

In [75]:
merged_df = merged_df.rename(columns={'White': 'white', 'Black': 'black', 'Hispanic': 'hispanic',
                                          'Asian/Native Hawaiian and Pacific Islander': 'asian/native hawaiian and pacific islander',
                                         'American Indian/Alaska Native': 'american indian/alaska native',
                                         'Multiple Races': 'multiple_races'})

In [78]:
merged_df

,id,year,state,white,black,hispanic,asian/native hawaiian and pacific islander,american indian/alaska native,multiple_races,total_poverty,n_killed,n_injured,population_rank,population,density_per_sq_mi
0,1,2016,Alabama,0.121,0.279,0.319,0.118,0.126,0.251,0.173,488,761,24,4887681,97.4271
1,2,2016,Alaska,0.068,0.000,0.178,0.000,0.274,0.104,0.106,88,103,49,735139,1.2694
2,3,2016,Arizona,0.110,0.211,0.238,0.136,0.360,0.155,0.165,277,294,14,7158024,66.2016
3,4,2016,Arkansas,0.138,0.288,0.255,0.129,0.263,0.254,0.172,190,351,34,3009733,58.3059
4,5,2016,California,0.096,0.223,0.190,0.113,0.226,0.132,0.144,1265,1896,1,39461588,254.2929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2017,Virginia,0.082,0.180,0.123,0.068,0.000,0.135,0.105,385,939,12,8501286,217.8776
96,97,2017,Washington,0.088,0.201,0.211,0.097,0.291,0.126,0.112,230,352,13,7523869,117.3249
97,98,2017,West Virginia,0.184,0.305,0.264,0.000,0.553,0.285,0.191,81,156,40,1804291,73.5443
98,99,2017,Wisconsin,0.083,0.292,0.205,0.142,0.266,0.209,0.109,209,519,21,5807406,108.0633


In [79]:
merged_df.to_sql(name="merged_data", con=engine, if_exists='append', index=False)